In [1]:
from langchain_community.graphs import Neo4jGraph
import pandas as pd
from pyprojroot import here
from urllib.parse import quote



In [2]:
NEO4J_URL = "bolt://localhost:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "Password"
NEO4J_DATABASE = 'neo4j'

In [3]:
graph = Neo4jGraph(url=NEO4J_URL, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE)

C:\Users\ombha\AppData\Local\Temp\ipykernel_19804\2317117226.py:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(url=NEO4J_URL, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE)


In [8]:
movie_csv_path = here("data/movie.csv")
movie_csv_path = movie_csv_path.as_posix()  # if it's a pathlib.Path object

print(pd.read_csv(movie_csv_path).columns)
print("DAta shape:", pd.read_csv(movie_csv_path).shape)

Index(['Unnamed: 0', 'movieId', 'released', 'title', 'actors', 'director',
       'genres', 'imdbRating', 'tagline'],
      dtype='object')
DAta shape: (20, 9)


In [9]:
pd.read_csv(movie_csv_path).head(3)

,Unnamed: 0,movieId,released,title,actors,director,genres,imdbRating,tagline
0,0,1,1995-11-22,Toy Story,Jim Varney|Tim Allen|Tom Hanks|Don Rickles,John Lasseter,Adventure|Animation|Children|Comedy|Fantasy,8.3,The adventure takes off!
1,1,2,1995-12-15,Jumanji,Robin Williams|Bradley Pierce|Kirsten Dunst|Jo...,Joe Johnston,Adventure|Children|Fantasy,6.9,Roll the dice and unleash the excitement!
2,2,3,1995-12-22,Grumpier Old Men,Walter Matthau|Ann-Margret|Jack Lemmon|Sophia ...,Howard Deutch,Comedy|Romance,6.6,Still Yelling. Still Fighting. Still Ready for...


In [11]:
graph.query("""
LOAD CSV WITH HEADERS FROM 'file:///' + $movie_directory    // Load CSV data from a file specified by $movie_directory
AS row                                                      // Each row in the CSV will be represented as 'row'
            
MERGE (m:Movie {id:row.movieId})                            // Merge a Movie node with the id from the row
SET m.released = date(row.released),                        // Set the 'released' property of the Movie node to the date from the row
    m.title = row.title,                                    // Set the 'title' property of the Movie node to the title from the row
    m.tagline = row.tagline,                                // Set the 'tagline' property of the Movie node to the tagline from the row
    m.imdbRating = toFloat(row.imdbRating)                  // Convert the 'imdbRating' from string to float and set it as the property
            
FOREACH (director in split(row.director, '|') |             // For each director in the list of directors from the row (split by '|') 
    MERGE (p:Person {name:trim(director)})                  // Merge a Person node with the director's name from the row, trimming any extra spaces
    MERGE (p)-[:DIRECTED]->(m))                             // Create a DIRECTED relationship from the director to the Movie
            
FOREACH (actor in split(row.actors, '|') |                  // For each actor in the list of actors from the row (split by '|') 
    MERGE (p:Person {name:trim(actor)})                     // Merge a Person node with the actor's name from the row, trimming any extra spaces
    MERGE (p)-[:ACTED_IN]->(m))                             // Create an ACTED_IN relationship from the actor to the Movie
            
FOREACH (genre in split(row.genres, '|') |                  // For each genre in the list of genres from the row (split by '|')
    MERGE (g:Genre {name:trim(genre)})                      // Merge a Genre node with the genre's name from the row, trimming any extra spaces
    MERGE (m)-[:IN_GENRE]->(g))                             // Create an IN_GENRE relationship from the Movie to the Genre
            
MERGE (l:Location {name:trim(row.location)})
MERGE (m)-[:WAS_TAKEN_IN]->(l)

MERGE (s:SimilarMovie {name:trim(row.similar_movie)})
MERGE (m)-[:IS_SIMILAR_TO]->(s)                                            
""",
params={"movie_directory": str(movie_csv_path)})            # Pass the parameter 'movie_directory' which contains the path to the CSV file

[]